# Libraries and Data

In [38]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

In [30]:
# Load the data
# YYYY-MM-DD
df = pd.read_csv('../nyc_data.csv', index_col = 0, parse_dates = True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [31]:
# Rename variable
df = df.rename({'Demand': 'y'}, axis = 1)
df

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077
...,...,...,...,...,...,...
2020-12-27,685.915026,0,0,0,2.89,38.674
2020-12-28,998.051170,0,0,0,8.83,166.712
2020-12-29,847.123399,0,0,0,3.48,161.865


# Prepare for LSTM

In [32]:
# Time Series object
series = TimeSeries.from_series(df.y) # Datapoint we are trying to predict
covariates = TimeSeries.from_dataframe(df.iloc[:, 1:])

In [33]:
# Year
year_series = datetime_attribute_timeseries(
    pd.date_range(start = series.start_time(),
                  freq = series.freq_str,
                  periods = df.shape[0]),
    attribute = "year",
    one_hot = False)

# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute = "month",
                                             one_hot = True)

# Weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute = "weekday",
                                               one_hot = True)

In [34]:
# Preparing scalers
transformer1 = Scaler()
transformer2 = Scaler()

In [35]:
# Scale the Y (timeseries)
y_transformed = transformer1.fit_transform(series)

In [36]:
# Sale the covariates
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

# LSTM (Long Short-Term Memory) Model

In [43]:
# Model
# https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
model = RNNModel(model = "LSTM",
                 hidden_dim = 20,
                 n_rnn_layers = 2,
                 dropout = 0,
                 n_epochs = 20,
                 optimizer_kwargs = {"lr": 0.003},
                 random_state = 1502,
                 training_length = 20,
                 input_chunk_length = 15, # Lower than the training length
#                  pl_trainer_kwargs = {"accelerator": "gpu", "gpus": -1,
#                                       "auto_select_gpus": True},
                 save_checkpoints = True)

In [44]:
# Fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.8 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params
0.031     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

C:\Users\Nes_X\Desktop\Time Series Forecasting Product\env\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:359: UserWarning: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
  warning_cache.warn(m)
`Trainer.fit` stopped: `max_epochs=20` reached.


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=2, dropout=0, training_length=20, n_epochs=20, optimizer_kwargs={'lr': 0.003}, random_state=1502, input_chunk_length=15, save_checkpoints=True)

In [ ]:
# Cross-validation
cv = model.historical_forecasts(y_transformed,
                                )